<a href="https://colab.research.google.com/github/mohammadAliMkh/Pytorch/blob/main/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/mohammadAliMkh/Pytorch.git
!mv "/content/Pytorch/food_vision_project" "/content/"
!rm -rf "/content/Pytorch/"
%cd /content/food_vision_project/

Cloning into 'Pytorch'...
remote: Enumerating objects: 493, done.
remote: Counting objects: 100% (362/362), done.
remote: Compressing objects: 100% (351/351), done.
remote: Total 493 (delta 29), reused 333 (delta 11), pack-reused 131
Receiving objects: 100% (493/493), 38.06 MiB | 25.62 MiB/s, done.
Resolving deltas: 100% (80/80), done.
/content/food_vision_project


In [10]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import torchvision
import torch
import matplotlib.pyplot as plt
import os
import shutil
import torchinfo

import data_setup

from pathlib import Path
from torchvision import transforms

In [4]:
dir = "/content/sample_data"

if os.path.exists(dir):
  shutil.rmtree(dir)
  print(dir + " deleted.")

/content/sample_data deleted.


In [5]:
efficient_weight = torchvision.models.EfficientNet_B0_Weights.DEFAULT
efficient_transformer = efficient_weight.transforms()
efficient_transformer

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [6]:
TRAIN_IMAGE_DIR = "/content/food_vision_project/pizza_steak_sushi/train"
TEST_IMAGE_DIR = "/content/food_vision_project/pizza_steak_sushi/test"

In [7]:
train_dataLoader , test_dataLoader , class_names = data_setup.create_dataLoader(TRAIN_IMAGE_DIR,
                             TEST_IMAGE_DIR,
                             batch_size = 32 , transformer = efficient_transformer)

In [8]:
efficient_model = torchvision.models.efficientnet_b0(weights = efficient_weight)
efficient_model

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-3dd342df.pth


  0%|          | 0.00/20.5M [00:00<?, ?B/s]

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [17]:
torchinfo.summary(efficient_model,
                  input_size = (1 , 3  ,224 , 224),
                  col_names = ["input_size" , "output_size", "num_params" , "trainable"],
                  col_width = 20 )

Layer (type:depth-idx)                                  Input Shape          Output Shape         Param #              Trainable
EfficientNet                                            [1, 3, 224, 224]     [1, 1000]            --                   True
├─Sequential: 1-1                                       [1, 3, 224, 224]     [1, 1280, 7, 7]      --                   True
│    └─Conv2dNormActivation: 2-1                        [1, 3, 224, 224]     [1, 32, 112, 112]    --                   True
│    │    └─Conv2d: 3-1                                 [1, 3, 224, 224]     [1, 32, 112, 112]    864                  True
│    │    └─BatchNorm2d: 3-2                            [1, 32, 112, 112]    [1, 32, 112, 112]    64                   True
│    │    └─SiLU: 3-3                                   [1, 32, 112, 112]    [1, 32, 112, 112]    --                   --
│    └─Sequential: 2-2                                  [1, 32, 112, 112]    [1, 16, 112, 112]    --                   True
│    